In [39]:
import torch 
from torch import nn
import torchvision
from torch.utils.data import DataLoader 
from torchvision import datasets 
from torchvision.transforms import ToTensor
import torchvision.transforms as transforms

In [5]:
training_data=datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 170498071/170498071 [00:55<00:00, 3052513.63it/s]


Extracting data\cifar-10-python.tar.gz to data


In [6]:
test_data=datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

Files already downloaded and verified


In [7]:
batch_size=64

train_dataloader=DataLoader(training_data,batch_size=batch_size)
test_dataloader=DataLoader(test_data,batch_size=batch_size)

for X,y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 3, 32, 32])
Shape of y: torch.Size([64]) torch.int64


In [40]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [9]:
device=(
    "cuda" 
    if torch.cuda.is_available()
    else "mps"  
    if torch.backends.mps.is_available()
    else "cpu" 
)
print(f"Using {device} device")

Using cpu device


In [19]:
class NeuralNetwork(nn.Module):
    def __init__(self):
      super().__init__()
      self.flatten=nn.Flatten()
      self.linear_relu_stack=nn.Sequential(
         nn.Linear(3*32*32,512),
         nn.ReLU(), 
         nn.Linear(512,512),
         nn.ReLU(),
         nn.Linear(512,10),
      )
    def forward(self,x):
       x=self.flatten(x)
       logits=self.linear_relu_stack(x)
       return logits 
model=NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=3072, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [30]:
import torch.nn.functional as F
class NeuralNetConv(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(3,6,5)
        self.pool=nn.MaxPool2d(2,2)
        self.conv2=nn.Conv2d(6,16,5)
        self.fc1=nn.Linear(16*5*5,120)
        self.fc2=nn.Linear(120,84)
        self.fc3=nn.Linear(84,10)
    def forward(self,x):
        x=self.pool(F.relu(self.conv1(x)))
        x=self.pool(F.relu(self.conv2(x)))
        x=torch.flatten(x,1)
        x=F.relu(self.fc1(X))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)

        return x  
net=NeuralNetConv()

In [32]:
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(net.parameters(),lr=1e-3,momentum=0.9)

In [33]:
def train(dataloader,model,loss,optimizer):
    size=len(dataloader.dataset)
    net.train()
    for batch,(X,y) in enumerate(dataloader):
        X,y=X.to(device),y.to(device)

        pred=model(X)
        loss=loss_fn(pred,y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch%100==0:
            loss,current=loss.item(),(batch+1)*len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [34]:
def test(dataloader,model,loss):
    size=len(dataloader.dataset)
    num_batches=len(dataloader)
    net.eval()

    test_loss,correct=0,0

    with torch.no_grad():
        for X,y in dataloader:
            X,y=X.to(device),y.to(device)
            pred=model(X)
            test_loss+=loss(pred,y).item()
            correct+=(pred.argmax(1)==y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [41]:
epochs = 2
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.018394  [   64/50000]
loss: 2.002177  [ 6464/50000]
loss: 1.774112  [12864/50000]
loss: 2.020839  [19264/50000]
loss: 1.973988  [25664/50000]
loss: 1.977074  [32064/50000]
loss: 2.076199  [38464/50000]
loss: 1.922954  [44864/50000]
Test Error: 
 Accuracy: 30.8%, Avg loss: 1.940486 

Epoch 2
-------------------------------
loss: 2.018394  [   64/50000]
loss: 2.002177  [ 6464/50000]
loss: 1.774112  [12864/50000]
loss: 2.020839  [19264/50000]
loss: 1.973988  [25664/50000]
loss: 1.977074  [32064/50000]
loss: 2.076199  [38464/50000]
loss: 1.922954  [44864/50000]
Test Error: 
 Accuracy: 30.8%, Avg loss: 1.940486 

Done!
